In [2]:
from el_toolkit.document import Document
from el_toolkit.lkb.lexical_knowledge_base import Lexical_Knowledge_Base
from transformers import BertTokenizer


lkb = Lexical_Knowledge_Base.read_json("data/knowledge_datasets/mesh_subset.json")

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                            do_lower_case=False, cache_dir=None)
                                            
doc = Document.read_json("/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/14_mentions.json")
doc.jupyter_display(lkb)
for doc in doc.segment(biomed_tokenizer,8):
  doc.jupyter_display(lkb)
  print("-----------------------------------")

Mentions Omitted: 0


-----------------------------------


-----------------------------------


In [2]:

from el_toolkit.entity_linkers.dual_encoder.featurizer import TrainFeaturizer


from el_toolkit.entity_linkers.dual_encoder.encoder import Entity


featurizer = DualEncoderTrainFeaturizer(biomed_tokenizer,entities,num_random_negatives=7)


In [3]:
featurizer.featurize([doc])

TypeError: list indices must be integers or slices, not str